In [56]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import statsmodels.api as sm
from statsmodels.stats.api import linear_rainbow
import warnings
warnings.filterwarnings("ignore")

Для построения моделей мы будем использовать датасет с фильмами, так как в датасете с картинками много пропусков, которые нельзя заменить (там пропуски носят смысловой характер). К сожалению, из-за этих ограничений мы не сможем провести анализ коэффициентов, чтобы понять, какие посты повышают ER (с фильмами или с картинками), но мы уже проверили это в гипотезах.

In [32]:
df = pd.read_csv('movies_dataset.csv')
df.head()

,likes,reposts,views,comments,attachments,text,movie,director,year,additional_info,...,globe_winner,movie_flg,log_likes,log_reposts,log_views,log_comments,reactions,ER,prize,other_award
0,46,30,5841,3,9,"«Чернокнижник: Армагеддон», 1993 год. \nРежисс...",Чернокнижник: Армагеддон,Энтони Хикокс,1993,NaN,...,0.0,True,3.850148,3.433987,8.672828,1.386294,79,1.352508,0.0,0.0
1,274,561,31437,3,9,"«Губная помада», 1976 год. \nРежиссер: Ламонт ...",Губная помада,Ламонт Джонсон,1976,"Марго Хемингуэй играет модель Крис, которая у...",...,0.0,True,5.616771,6.331502,10.355773,1.386294,838,2.665649,0.0,0.0
2,102,82,13583,3,9,"«Соблазн», 2001 год. \nРежиссер: Майкл Кристофер.",Соблазн,Майкл Кристофер,2001,NaN,...,0.0,True,4.634729,4.418841,9.516648,1.386294,187,1.376721,0.0,0.0
3,348,132,26569,1,9,"«Красота по-американски», 1999 год. \nРежиссер...",Красота по-американски,Сэм Мендес,1999,NaN,...,1.0,True,5.855072,4.890349,10.187538,0.693147,481,1.810381,2.0,1.0
4,512,172,32902,8,9,"«Дневник баскетболиста», 1995 год. \nРежиссер:...",Дневник баскетболиста,Скотт Кэлверт,1995,NaN,...,0.0,True,6.240276,5.153292,10.401319,2.197225,692,2.103216,0.0,0.0


Начнём с построения линейной регрессии.

In [33]:
X = df[['attachments', 'year', 'oscar_winner', 'oscar_nominee', 'palme_winner', 'globe_winner', 'prize', 'other_award']]
y = df['ER']
#делим выборку на обучающую и тестовую в соотношении 80-20
(X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size = .2, random_state = 42)
y_train = np.array(y_train).reshape(-1, 1)
lin_model = LinearRegression()
lin_model.fit(X_train, y_train)
y_test_pred = lin_model.predict(X_test)
y_train_pred = lin_model.predict(X_train)

Посмотрим на основные метрики линейной регрессии и на коэффициенты.

In [34]:
mse_linear = mean_squared_error(y_true=y_test, y_pred=y_test_pred)
mse_linear

0.40427753032553376

In [60]:
mae_linear = mean_absolute_error(y_true=y_test, y_pred=y_test_pred)
mae_linear

0.49075587327075726

In [61]:
r2_linear = r2_score(y_true=y_test, y_pred=y_test_pred)
r2_linear

0.03792896729010031

Как мы видим, несмотря на низкие значения MSE и MAE, модель очень неточно предсказывает значение ER. Это связано с тем, что ER сама по себе метрика с маленьким значением (имеет значение от 0.5 до 4), поэтому отклонения от нее тоже небольшие, MSE получился даже меньше MAE, так как большинство значений лежат в диапозоне 0.5-1, соответственно, от возведения в квадрат ошибка только уменьшается. Тем не менее, эта модель плохо подходит для прогнозирования.

In [62]:
lin_model.coef_

array([[-0.01268605,  0.00868737, -0.00340978, -0.08042231, -0.22133332,
         0.26005043,  0.03530733,  0.03871711]])

Коэффициенты также получились очень маленькими (можем связать это с маленьким значением самого коэффициента). Из интересного можно отметить, что самыми большими по модулю оказались коэффициенты перед palme_winner и globe_winner. Если globe_winner идет с плюсом, что говорит нам о возможной каузации (что наличие у фильма "Золотого Глобуса" влияет на ER положительно), то вот "Пальмовая ветвь", наоборот, даже уменьшает ER.

Для того, чтобы более точно судить о причинно-следственных связях, построим также Лассо-модель и посмотрим, улучшит ли она наш прогноз.

In [63]:
lasso_cv = LassoCV(cv=5)
lasso_cv.fit(X_train, y_train)
alpha = lasso_cv.alpha_
alpha

0.006169241074241648

In [64]:
lasso_model = Lasso(alpha)
lasso_model.fit(X_train, y_train)
y_test_pred_lasso = lasso_model.predict(X_test)

In [65]:
r2_lasso = r2_score(y_true=y_test, y_pred=y_test_pred_lasso)
r2_lasso

0.026045543594203635

К сожалению, Лассо-модель не улучшила наши прогнозы по сравнению с линейной регрессией. Посмотрим, что случилось с коэффициентами.

In [66]:
lasso_model.coef_

array([-0.        ,  0.63680436,  0.        , -0.        , -0.        ,
        0.        ,  0.        ,  0.        ])

Ухудшение прогноза связано с обнулением практически всех коэффициентов. Можно сказать, что с точки зрения Лассо-регрессии наиболее значимым оказался год выпуска, перед остальными переменными коэффициенты обнулились. Этот результат радует, так как именно с годом выпуска мы предполагали наличие связи у целевой переменной.

Так как качество прогнозирования достаточно низкое, мы предполагаем, что линейная форма модели не отражает реальные данные. Проведем тест Рамсея, чтобы проверить эту гипотезу. Создадим новый датасет, в который добавим корни, квадраты и кубы независимых переменных (корень нужен, так как корреляционное облако таргета и переменной year похоже на квадратный корень).

In [67]:
df_ramsey = df[['ER', 'attachments', 'year', 'oscar_winner', 'oscar_nominee', 'palme_winner', 'globe_winner', 'prize', 'other_award']]
for i in ['attachments', 'year', 'oscar_winner', 'oscar_nominee', 'palme_winner', 'globe_winner', 'prize', 'other_award']:
    df_ramsey[f'{i}_sqrt'] = np.sqrt(df_ramsey[i])
    df_ramsey[f'{i}_sqrd'] = df_ramsey[i]**2
    df_ramsey[f'{i}_cube'] = df_ramsey[i]**3
df_ramsey.head()

,ER,attachments,year,oscar_winner,oscar_nominee,palme_winner,globe_winner,prize,other_award,attachments_sqrt,...,palme_winner_cube,globe_winner_sqrt,globe_winner_sqrd,globe_winner_cube,prize_sqrt,prize_sqrd,prize_cube,other_award_sqrt,other_award_sqrd,other_award_cube
0,1.352508,9,1993,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,2.665649,9,1976,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,1.376721,9,2001,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,1.810381,9,1999,1.0,1.0,0.0,1.0,2.0,1.0,3.0,...,0.0,1.0,1.0,1.0,1.414214,4.0,8.0,1.0,1.0,1.0
4,2.103216,9,1995,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


Теперь построим линейную модель с новыми признаками.

In [68]:
#создаем список со всеми переменными, кроме таргета
columns = df_ramsey.columns.tolist() 
columns.remove('ER')

In [69]:
X = df_ramsey[columns]
y = df_ramsey['ER']
(X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size = .2, random_state = 42)
y_train = np.array(y_train).reshape(-1, 1)
lin_model_extended = LinearRegression()
lin_model_extended.fit(X_train, y_train)
y_test_pred_extended = lin_model_extended.predict(X_test)
y_train_pred_extended = lin_model_extended.predict(X_train)

In [70]:
mse_linear_extended = mean_squared_error(y_true=y_test, y_pred=y_test_pred_extended)
mse_linear_extended

0.3586837215130129

In [71]:
r2_linear_extended = r2_score(y_true=y_test, y_pred=y_test_pred_extended)
r2_linear_extended

0.146429884207543

Даже без проведения дополнительного тестирования видно, что качество предсказания повысилось - MSE уменьшилось, а $R^2$ вырос, что говорит о том, что модель описывает большее количество данных.

In [72]:
coef = (lin_model_extended.coef_).tolist()

In [73]:
for i in range(len(columns)):
    print (f'Переменная {columns[i]} = {coef[0][i]}')

Переменная attachments = -14.708751940289567
Переменная year = -8427.695151301123
Переменная oscar_winner = 9.49710580089299
Переменная oscar_nominee = 0.027315276701763973
Переменная palme_winner = 0.24355708836029782
Переменная globe_winner = 0.3463434611935889
Переменная prize = -1.3567424400709507
Переменная other_award = 0.6133214191009134
Переменная attachments_sqrt = 23.473561087506702
Переменная attachments_sqrd = 1.5194567796448777
Переменная attachments_cube = -0.07233835162324552
Переменная year_sqrt = 398112.9139379447
Переменная year_sqrd = 1.4322612371410632
Переменная year_cube = -0.000146031523253221
Переменная oscar_winner_sqrt = -2.0078593091602963
Переменная oscar_winner_sqrd = -2.0078593091602963
Переменная oscar_winner_cube = -2.0078593091602963
Переменная oscar_nominee_sqrt = -0.0302136698994718
Переменная oscar_nominee_sqrd = -0.0302136698994718
Переменная oscar_nominee_cube = -0.0302136698994718
Переменная palme_winner_sqrt = 0.2470349120808131
Переменная palme_

Посмотрим, какие из 4 коэффициентов (обычный, корень, квадрат, куб) самые большие по модулю для каждой переменной. Attachment - sqrt, year - sqrt, oscar_winner - обычный показатель, oscar_nominee - sqrt, sqrd и cube имеют одинаковые значения, возьмем sqrt. Это верно и для palme_winner, и для globe_winner, возьмем в обоих случаях значение sqrt. Для переменной prize возьмем sqrt (самое большое по модулю), для other_award - cube. Построим линейную модель с использованием только этих переменных и проверим, повысится ли ее точность.

In [74]:
X = df_ramsey[['attachments_sqrt', 'year_sqrt', 'oscar_winner', 'oscar_nominee_sqrt', 'palme_winner_sqrt', 'globe_winner_sqrt', 'prize_sqrt', 'other_award_cube']]
y = df_ramsey['ER']
(X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size = .2, random_state = 42)
y_train = np.array(y_train).reshape(-1, 1)
lin_model_optimal = LinearRegression()
lin_model_optimal.fit(X_train, y_train)
y_test_pred_optimal = lin_model_optimal.predict(X_test)
y_train_pred_optimal = lin_model_optimal.predict(X_train)

In [78]:
mse_linear_optimal = mean_squared_error(y_true=y_test, y_pred=y_test_pred_optimal)
mse_linear_optimal

0.4015398691283726

In [79]:
r2_linear_optimal = r2_score(y_true=y_test, y_pred=y_test_pred_optimal)
r2_linear_optimal

0.04444385950546004

К сожалению, попытка подобрать коэффициенты не оказалась успешной, точность модели снизилась (но она немного выше, чем была изначально). Однако мы точно убедились, что линейная модель не сильно подходит для предсказания и необходимо искать более сложные зависимости между данными.

In [80]:
#еще раз построим линейную регрессию с помощью другой библиотеки для проверки данных на постоянство дисперсии остатков
X = df_ramsey[columns]
y = df_ramsey['ER']
X = sm.add_constant(X)
linear_model_2 = sm.OLS(y, X).fit()
test_result = linear_rainbow(linear_model_2)
test_result[1]>0.05

False

Гипотеза о постоянстве остатков дисперсий отвергается на уровне значимости 5%. Важно отметить, что это не говорит о 100% нелинейной зависимости, так как этот тест чувствителен к выбросам и гомоскедатичность может вызываться не только линейностью данных 